In [1]:
import torch
from typing import Literal
# they could just copy the pervious token, on another dim?
class MSGPT(torch.nn.Module):
    def __init__(self, backbone: Literal["transformer", "LSTM", "GRU"] = "transformer" ):
        super().__init__()
        if not backbone in ["transformer", "LSTM", "GRU"]:
            raise ValueError("backbone must be one of 'transformer', 'LSTM', 'GRU'")
        self.begin = torch.nn.Embedding(1, 1024)
        self.backbone_type = backbone
        if backbone == "transformer":
            self.pos = torch.nn.Embedding(1024, 1024)
            self.backbone = torch.nn.TransformerEncoder(torch.nn.TransformerDecoderLayer(d_model=1024, nhead=16, batch_first=True), num_layers=8, enable_nested_tensor=False)
        elif backbone == "LSTM":
            self.backbone = torch.nn.LSTM(input_size=1024, hidden_size=1024, num_layers=8, batch_first=True)
        else:
            self.backbone = torch.nn.GRU(input_size=1024, hidden_size=1024, num_layers=8, batch_first=True)
    def forward(self, input):
        # input [B, L, D]
        # begin = self.begin(torch.zeros(1, device=input.device, dtype=torch.long)).unsqueeze(0)
        # input = torch.cat([begin, input], dim=1)
        if self.backbone_type == "transformer":
            # input = self.pos(torch.arange(input.shape[1], device=input.device)) + input 
            print(input.shape)
            return self.backbone(input)
        else:
            return self.backbone(input)[0]


In [4]:
model = MSGPT("transformer")
model = model.train()
model(torch.randn(1, 10, 1024))

torch.Size([1, 10, 1024])


TypeError: forward() got an unexpected keyword argument 'src_mask'